新增提早停止
目前沒有切train and test data


In [3]:
import pandas as pd
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn as dglnn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import pickle
import random
import numpy as np


In [4]:
import torch

# 检查 CUDA 是否可用
cuda_available = torch.cuda.is_available()

# 输出 CUDA 是否可用
print("CUDA available:", cuda_available)

# 如果 CUDA 可用，打印当前 CUDA 设备的名称
if cuda_available:
    print("Current CUDA device:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA available: True
Current CUDA device: NVIDIA GeForce RTX 4060 Ti


In [5]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 数据集类定义
class BPRDataset(Dataset):
    def __init__(self, user_item_pairs, total_items):
        self.user_item_pairs = user_item_pairs
        self.total_items = total_items

    def __len__(self):
        return len(self.user_item_pairs)

    def __getitem__(self, idx):
        user, pos_item = self.user_item_pairs[idx]
        neg_item = np.random.randint(0, self.total_items)
        while neg_item == pos_item:
            neg_item = np.random.randint(0, self.total_items)
        return user, pos_item, neg_item
# Path to your saved graph
graph_path = r'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\graph\hetero_graph03_with_V_T_A_features.pkl'

# Load the graph
with open(graph_path, 'rb') as f:
    graph = pickle.load(f)
# 数据路径
data_path = 'D:\\CODE\\multi-model knowledge graph multi-graph recommendation system\\data\\cleanuser_rating.csv'
df = pd.read_csv(data_path)
total_items = df['movieId'].max() + 1  # 确保这是正确的电影总数
print("Total items (movies):", total_items)
# 将用户ID和电影ID编码为连续整数
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df['userId'] = user_encoder.fit_transform(df['userId'])
df['movieId'] = item_encoder.fit_transform(df['movieId'])
num_users = df['userId'].nunique()  # 计算唯一的用户数量
num_items = df['movieId'].nunique()  # 计算唯一的电影数量
# 分割数据为训练集和验证集
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# 转换为用户-项目配对
train_pairs = list(zip(train_df['userId'].values, train_df['movieId'].values))
valid_pairs = list(zip(valid_df['userId'].values, valid_df['movieId'].values))

# 总电影数
total_items = df['movieId'].max() + 1

# 创建数据集
train_dataset = BPRDataset(train_pairs, total_items)
valid_dataset = BPRDataset(valid_pairs, total_items)

# 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

try:
    for i, (user_ids, pos_item_ids, neg_item_ids) in enumerate(train_loader):
        print(f"Batch {i}: User IDs: {user_ids} Pos IDs: {pos_item_ids} Neg IDs: {neg_item_ids}")
        if i >= 5:  # 只打印前5批次以限制输出
            break
except Exception as e:
    print(f"An error occurred: {e}")


Total items (movies): 131263
Batch 0: User IDs: tensor([ 5137,   168, 11600,  8015,  2106,  9236,  3945,  1860,  3022,  5518,
        10888,   799,  1936,  6034, 10440,  7771,  2091,  5683, 10333,  4283,
        11733,  2928,  4490, 10431,  3343, 11656,  2695,  6770,  7391,  2802,
         2631,  5601,  2964, 10721,  6456,  8063,  1323,  8598,  1175,  5796,
         7016,  5857,  5762, 10428,  1355,  1253,  5115,  3761,  8420, 11995,
         4043,  6339,   853, 10278,  2703,  8380, 12019,  4636,  4986,  8074,
         6234,  6725, 11714,  8529,  4142,  3503, 10569,  9038,  1248,  9645,
         9323,  7621, 11447, 11030,  9650,  8604,   386,  4169,  4525,   242,
          935,    83, 11270,  6850,  9057,  2452,   746,  8049, 11493,  7896,
         6433,  9183, 11057,  1150, 11445, 11047,  3566,  1726,  7230,   803,
         1766,  9871,  7858, 10220,  8860,  1766,  7394, 11558,  4463, 10483,
        11399,   941,  5714,    60,  3846,  4611,  2949,  3110,  8844, 11958,
        12032,  

In [6]:
import torch
import pickle
import dgl

# Load your graph
with open('D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\graph\hetero_graph03_with_V_T_A_features.pkl', 'rb') as f:
    hetero_graph = pickle.load(f)

# Function to check features for a specific node type
def check_features(graph, node_type):
    if node_type in graph.ntypes:
        if 'features' in graph.nodes[node_type].data:
            features = graph.nodes[node_type].data['features']
            print(f"Features are available in '{node_type}' nodes.")
            print(f"Shape of the features: {features.shape}")
            print(f"Average of features: {torch.mean(features)}")
            print(f"Non-zero elements in features: {torch.count_nonzero(features)}")
            if node_type == 'movietext' and features.shape[1] != 128:  # Adjusted expected dimension
                print(f"Incorrect dimension of features for {node_type}: {features.shape[1]}")
            elif node_type == 'movieimage' and features.shape[1] != 2048:  # Expected dimension for image
                print(f"Incorrect dimension of features for {node_type}: {features.shape[1]}")
            elif node_type == 'movieaudio' and features.shape[1] != 128:  # Expected dimension for audio
                print(f"Incorrect dimension of features for {node_type}: {features.shape[1]}")
        else:
            print(f"No features data found in '{node_type}' nodes.")
    else:
        print(f"No '{node_type}' node type found in the graph.")

# Check features for each modality
check_features(hetero_graph, 'movietext')
check_features(hetero_graph, 'movieimage')
check_features(hetero_graph, 'movieaudio')

# Optionally, save the graph again if modifications are made
# with open('hetero_graph_with_updated_features.pkl', 'wb') as f:
#     pickle.dump(hetero_graph, f)

# If you need to reload and check the graph, uncomment these lines
# with open('hetero_graph_with_updated_features.pkl', 'rb') as f:
#     loaded_graph = pickle.load(f)


Features are available in 'movietext' nodes.
Shape of the features: torch.Size([5996, 384])
Average of features: 0.0
Non-zero elements in features: 0
Incorrect dimension of features for movietext: 384
Features are available in 'movieimage' nodes.
Shape of the features: torch.Size([5996, 2048])
Average of features: 0.25277090072631836
Non-zero elements in features: 8816537
Features are available in 'movieaudio' nodes.
Shape of the features: torch.Size([5996, 128])
Average of features: 0.0
Non-zero elements in features: 0


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dgl.function as fn
from dgl.nn import GraphConv, GATConv
import dgl
import dgl.nn as dglnn
import pickle

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
import dgl

class HeteroGraphConv(nn.Module):
    def __init__(self, num_users, num_items, feature_sizes, embedding_dim=128, id_embedding_size=16, num_heads=4, num_layers=4, dropout=0.5):
        super(HeteroGraphConv, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        # Embedding layers for users and items
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)

        # Embedding layers for ID-based nodes, including production team members
        self.production_team_nodes = [
            'actor', 'actress', 'composer', 'director', 'editor', 
            'producer', 'writer'
        ]
        self.embeddings = nn.ModuleDict({
            node_type: nn.Embedding(10000, id_embedding_size) for node_type in feature_sizes if node_type not in ['movie', 'movieimage', 'movietext', 'movieaudio']
        })

        # Transformation layers for modal-specific features
        self.image_transform = nn.Linear(feature_sizes['movieimage'], 32)
        self.text_transform = nn.Linear(feature_sizes['movietext'], 32)
        self.audio_transform = nn.Linear(feature_sizes['movieaudio'], 32)

        # GAT layers for each modality
        self.gat_layers = nn.ModuleDict({
            'movieimage': nn.ModuleList([GATConv(32, 32, num_heads=num_heads) for _ in range(num_layers)]),
            'movietext': nn.ModuleList([GATConv(32, 32, num_heads=num_heads) for _ in range(num_layers)]),
            'movieaudio': nn.ModuleList([GATConv(32, 32, num_heads=num_heads) for _ in range(num_layers)])
        })

        # GAT layers for production team members
        for node in self.production_team_nodes:
            self.gat_layers[node] = nn.ModuleList([GATConv(id_embedding_size, 32, num_heads=num_heads) for _ in range(num_layers)])

        # Attention layer for feature aggregation
        self.attention = nn.MultiheadAttention(embed_dim=32 * num_heads, num_heads=num_heads, dropout=dropout)

        # Additional layer to process concatenated features
        self.fc = nn.Linear(32 * (3 + len(self.production_team_nodes)) * num_heads, 128)  # Adjust the size according to the output of GAT

    def forward(self, user_ids, item_ids, graph=None, features=None):
        user_emb = self.user_embeddings(user_ids)
        item_emb = self.item_embeddings(item_ids)

        if graph is not None and features is not None:
            # Process modal-specific features if provided
            image_features = F.relu(self.image_transform(features['movieimage']))
            text_features = F.relu(self.text_transform(features['movietext']))
            audio_features = F.relu(self.audio_transform(features['movieaudio']))

            for layer in self.gat_layers['movieimage']:
                image_features = layer(graph, image_features).flatten(1)
                image_features = F.dropout(image_features, self.dropout, training=self.training)
            for layer in self.gat_layers['movietext']:
                text_features = layer(graph, text_features).flatten(1)
                text_features = F.dropout(text_features, self.dropout, training=self.training)
            for layer in self.gat_layers['movieaudio']:
                audio_features = layer(graph, audio_features).flatten(1)
                audio_features = F.dropout(audio_features, self.dropout, training=self.training)

            # Process production team members' features
            production_features = []
            for node_type in self.production_team_nodes:
                team_features = self.embeddings[node_type](graph.nodes[node_type].data['feat'])
                for layer in self.gat_layers[node_type]:
                    team_features = layer(graph, team_features).flatten(1)
                    team_features = F.dropout(team_features, self.dropout, training=self.training)
                production_features.append(team_features)

            # Concatenate all features
            combined_features = torch.cat([image_features, text_features, audio_features] + production_features, dim=1)
            combined_features = F.relu(self.fc(combined_features))
            combined_features = F.dropout(combined_features, self.dropout, training=self.training)

            # Apply attention mechanism
            combined_features = combined_features.unsqueeze(0)
            attn_output, _ = self.attention(combined_features, combined_features, combined_features)
            combined_features = attn_output.squeeze(0)

            # Symmetric normalization
            user_deg = graph.in_degrees(user_ids).float().clamp(min=1)
            item_deg = graph.in_degrees(item_ids).float().clamp(min=1)
            norm = torch.sqrt(user_deg.unsqueeze(1) * item_deg.unsqueeze(1))

            combined_features = combined_features / norm

            return user_emb, item_emb, combined_features
        return user_emb, item_emb, None


In [9]:
def bpr_loss(users, pos_items, neg_items, lambda_reg, model):
    # 獲取用戶和項目的嵌入
    user_embeddings = model.embedding_user(users)
    pos_item_embeddings = model.embedding_item(pos_items)
    neg_item_embeddings = model.embedding_item(neg_items)
    
    # 計算用戶對正樣本和負樣本的偏好預測
    pos_scores = torch.sum(user_embeddings * pos_item_embeddings, dim=1)
    neg_scores = torch.sum(user_embeddings * neg_item_embeddings, dim=1)
    
    # 計算 BPR 損失
    loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))
    
    # 添加 L2 正則化
    reg_loss = lambda_reg * (user_embeddings.norm(2).pow(2) + 
                             pos_item_embeddings.norm(2).pow(2) +
                             neg_item_embeddings.norm(2).pow(2))
    
    return loss + reg_loss




In [10]:
import torch
def dcg_at_k(scores, k=5):
    ranks = torch.log2(torch.arange(2, k+2).float()).to(scores.device)  # Log term in DCG formula
    return (scores[:k] / ranks).sum()  # Only consider the top k scores


def ndcg_at_k(predicted_scores, true_relevance, k=5):
    _, indices = torch.sort(predicted_scores, descending=True)
    true_sorted_by_pred = true_relevance[indices]
    ideal_sorted, _ = torch.sort(true_relevance, descending=True)

    dcg = dcg_at_k(true_sorted_by_pred[:k])
    idcg = dcg_at_k(ideal_sorted[:k])
    return (dcg / idcg).item() if idcg > 0 else 0.0


之後要改成ndcg的版本


In [11]:
# import torch.optim as optim
# import torch



# def train_model(model, train_loader, epochs, lambda_reg, optimizer, validation_loader=None):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0

#         # Train with BPR loss
#         for user_ids, pos_item_ids, neg_item_ids in train_loader:
#             optimizer.zero_grad()

#             user_embeddings, pos_item_embeddings = model(user_ids, pos_item_ids)
#             _, neg_item_embeddings = model(user_ids, neg_item_ids)
            
#             # BPR Loss
#             pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
#             neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
#             loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))
            
#             reg_loss = lambda_reg * (user_embeddings.norm(2).pow(2) + 
#                                      pos_item_embeddings.norm(2).pow(2) +
#                                      neg_item_embeddings.norm(2).pow(2))
#             total_loss_val = loss + reg_loss
#             total_loss_val.backward()
#             optimizer.step()
#             total_loss += total_loss_val.item()

#         print(f'Epoch {epoch+1}: Average Training Loss: {total_loss / len(train_loader)}')

#         # Evaluate using NDCG
#         if validation_loader:
#             model.eval()
#             with torch.no_grad():
#                 all_predicted_scores = torch.Tensor()
#                 all_true_scores = torch.Tensor()
#                 for user_ids, pos_item_ids, neg_item_ids in validation_loader:
#                     user_embeddings, pos_item_embeddings = model(user_ids, pos_item_ids)
#                     _, neg_item_embeddings = model(user_ids, neg_item_ids)
                    
#                     pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
#                     neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
                    
#                     scores = pos_scores - neg_scores
#                     true_relevance = torch.ones_like(scores)  # assuming all positive items should be ranked higher
                    
#                     all_predicted_scores = torch.cat((all_predicted_scores, scores), 0)
#                     all_true_scores = torch.cat((all_true_scores, true_relevance), 0)
                
#                 ndcg_value = ndcg_at_k(all_predicted_scores, all_true_scores, k=5)
#                 print(f'Epoch {epoch+1}: Validation NDCG@5 = {ndcg_value:.4f}')



In [12]:
# def train_model(model, train_loader, epochs, lambda_reg, optimizer, device, validation_loader=None):
#     model.to(device)  # 确保模型在正确的设备上
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0

#         for user_ids, pos_item_ids, neg_item_ids in train_loader:
#             user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)
            
#             optimizer.zero_grad()
#             user_embeddings, pos_item_embeddings = model(user_ids, pos_item_ids)
#             _, neg_item_embeddings = model(user_ids, neg_item_ids)
            
#             # BPR Loss
#             pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
#             neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
#             loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))
            
#             reg_loss = lambda_reg * (user_embeddings.norm(2).pow(2) +
#                                      pos_item_embeddings.norm(2).pow(2) +
#                                      neg_item_embeddings.norm(2).pow(2))
#             total_loss_val = loss + reg_loss
#             total_loss_val.backward()
#             optimizer.step()
#             total_loss += total_loss_val.item()

#         print(f'Epoch {epoch+1}: Average Training Loss: {total_loss / len(train_loader)}')

#         if validation_loader:
#             evaluate_metrics(model, validation_loader)
# def evaluate_metrics(model, validation_loader, k=5):
#     device = next(model.parameters()).device  # 获取模型参数所在的设备
#     model.eval()
#     total_dcg = 0
#     total_ndcg = 0
#     total_recall = 0
#     total_relevant_items = 0

#     with torch.no_grad():
#         for data in validation_loader:
#             user_ids, pos_item_ids, neg_item_ids = data
#             user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)
            
#             user_embeddings, pos_item_embeddings = model(user_ids, pos_item_ids)
#             _, neg_item_embeddings = model(user_ids, neg_item_ids)
            
#             pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
#             neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
            
#             scores = torch.cat((pos_scores, neg_scores))
#             labels = torch.cat((torch.ones_like(pos_scores), torch.zeros_like(neg_scores)))  # 真实标签
            
#             # 计算分数并排序
#             _, indices = torch.sort(scores, descending=True)
#             sorted_labels = labels[indices]
            
#             # 使用已定义的函数计算 DCG 和 NDCG
#             dcg_value = dcg_at_k(sorted_labels, k)
#             idcg_value = dcg_at_k(torch.ones(k).to(device), k)  # 理想情况下的排序
#             ndcg_value = dcg_value / max(idcg_value, 1e-10)  # 避免除以零
#             recall_value = sorted_labels[:k].sum() / labels.sum()

#             total_dcg += dcg_value
#             total_ndcg += ndcg_value
#             total_recall += recall_value
#             total_relevant_items += labels.sum()

#     average_dcg = total_dcg / len(validation_loader)
#     average_ndcg = total_ndcg / len(validation_loader)
#     average_recall = total_recall / len(validation_loader)
#     print(f'Average DCG@{k}: {average_dcg:.4f}')
#     print(f'Average NDCG@{k}: {average_ndcg:.4f}')
#     print(f'Average Recall@{k}: {average_recall:.4f}')

In [13]:
import torch

def train_model(model, train_loader, epochs, lambda_reg, optimizer, device, validation_loader=None, patience=5):
    model.to(device)  # 确保模型在正确的设备上
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for user_ids, pos_item_ids, neg_item_ids in train_loader:
            user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)
            
            optimizer.zero_grad()
            user_embeddings, pos_item_embeddings, _ = model(user_ids, pos_item_ids)
            _, neg_item_embeddings, _ = model(user_ids, neg_item_ids)
            
            # BPR Loss
            pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
            neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
            loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))
            
            reg_loss = lambda_reg * (user_embeddings.norm(2).pow(2) +
                                     pos_item_embeddings.norm(2).pow(2) +
                                     neg_item_embeddings.norm(2).pow(2))
            total_loss_val = loss + reg_loss
            total_loss_val.backward()
            optimizer.step()
            total_loss += total_loss_val.item()

        print(f'Epoch {epoch+1}: Average Training Loss: {total_loss / len(train_loader)}')

        if validation_loader:
            val_loss = evaluate_metrics(model, validation_loader)
            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f'Early stopping triggered after {epoch+1} epochs')
                break

def evaluate_metrics(model, validation_loader, k=5):
    device = next(model.parameters()).device  # 获取模型参数所在的设备
    model.eval()
    total_dcg = 0
    total_ndcg = 0
    total_recall = 0
    total_relevant_items = 0
    total_val_loss = 0

    with torch.no_grad():
        for data in validation_loader:
            user_ids, pos_item_ids, neg_item_ids = data
            user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)
            
            user_embeddings, pos_item_embeddings, _ = model(user_ids, pos_item_ids)
            _, neg_item_embeddings, _ = model(user_ids, neg_item_ids)
            
            pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
            neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
            
            scores = torch.cat((pos_scores, neg_scores))
            labels = torch.cat((torch.ones_like(pos_scores), torch.zeros_like(neg_scores))).to(device)  # 真实标签
            
            # 计算验证损失
            val_loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))
            total_val_loss += val_loss.item()

            # 计算分数并排序
            _, indices = torch.sort(scores, descending=True)
            sorted_labels = labels[indices]
            
            # 使用已定义的函数计算 DCG 和 NDCG
            dcg_value = dcg_at_k(sorted_labels, k)
            idcg_value = dcg_at_k(torch.ones(k).to(device), k)  # 理想情况下的排序
            ndcg_value = dcg_value / max(idcg_value, 1e-10)  # 避免除以零
            recall_value = sorted_labels[:k].sum() / labels.sum()

            total_dcg += dcg_value
            total_ndcg += ndcg_value
            total_recall += recall_value
            total_relevant_items += labels.sum()

    average_dcg = total_dcg / len(validation_loader)
    average_ndcg = total_ndcg / len(validation_loader)
    average_recall = total_recall / len(validation_loader)
    average_val_loss = total_val_loss / len(validation_loader)
    print(f'Average DCG@{k}: {average_dcg:.4f}')
    print(f'Average NDCG@{k}: {average_ndcg:.4f}')
    print(f'Average Recall@{k}: {average_recall:.4f}')
    print(f'Validation Loss: {average_val_loss:.4f}')
    return average_val_loss

def dcg_at_k(sorted_labels, k):
    sorted_labels = sorted_labels[:k]
    gains = 2 ** sorted_labels - 1
    discounts = torch.log2(torch.arange(2, k + 2).float().to(sorted_labels.device))  # 将discounts放到相同设备上
    return (gains / discounts).sum()

def dcg_at_k(sorted_labels, k):
    sorted_labels = sorted_labels[:k]
    gains = 2 ** sorted_labels - 1
    discounts = torch.log2(torch.arange(2, k + 2).float().to(sorted_labels.device))  # 将discounts放到相同设备上
    return (gains / discounts).sum()




# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_sizes = {
    'movie': 2560,    # 如果电影节点存储所有多媒体特征的总和
    'actor': 50,      # 演员的嵌入维度
    'actress': 50,    # 女演员的嵌入维度
    'director': 50,   # 导演的嵌入维度
    'producer': 50,   # 制片的嵌入维度
    'movieimage': 2048,  # 图像特征维度
    'movietext': 384,    # 文字特征维度
    'movieaudio': 128,   # 音频特征维度
    'user': 100         # 假设用戶的嵌入维度，可能需要根据用户数量和系统复杂性进行调整
}
# 初始化和配置模型，加载数据，然后开始训练和评估
model = HeteroGraphConv(num_users, num_items, feature_sizes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, 7, 0.01, optimizer, device, valid_loader)

# 保存模型
torch.save(model.state_dict(), 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth')

# 加载模型
model.load_state_dict(torch.load('D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth'))
model.eval()


d:\Anaconda\envs\GNN\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1: Average Training Loss: 299.9722746115699
Average DCG@5: 0.3842
Average NDCG@5: 0.1303
Average Recall@5: 0.0054
Validation Loss: 2.3830
Epoch 2: Average Training Loss: 116.72475050437023
Average DCG@5: 0.1888
Average NDCG@5: 0.0640
Average Recall@5: 0.0026
Validation Loss: 1.1668
Epoch 3: Average Training Loss: 50.982290731324056
Average DCG@5: 0.1439
Average NDCG@5: 0.0488
Average Recall@5: 0.0021
Validation Loss: 0.7992
Epoch 4: Average Training Loss: 24.057049934413843
Average DCG@5: 0.1199
Average NDCG@5: 0.0407
Average Recall@5: 0.0016
Validation Loss: 0.7128
Epoch 5: Average Training Loss: 12.522783073256589
Average DCG@5: 0.1139
Average NDCG@5: 0.0386
Average Recall@5: 0.0017
Validation Loss: 0.6959
Epoch 6: Average Training Loss: 7.224949376403436
Average DCG@5: 0.4017
Average NDCG@5: 0.1363
Average Recall@5: 0.0064
Validation Loss: 0.6935
Epoch 7: Average Training Loss: 4.539979433992556
Average DCG@5: 0.8050
Average NDCG@5: 0.2730
Average Recall@5: 0.0121
Validation L

HeteroGraphConv(
  (user_embeddings): Embedding(12171, 128)
  (item_embeddings): Embedding(5996, 128)
  (embeddings): ModuleDict(
    (actor): Embedding(10000, 16)
    (actress): Embedding(10000, 16)
    (director): Embedding(10000, 16)
    (producer): Embedding(10000, 16)
    (user): Embedding(10000, 16)
  )
  (image_transform): Linear(in_features=2048, out_features=32, bias=True)
  (text_transform): Linear(in_features=384, out_features=32, bias=True)
  (audio_transform): Linear(in_features=128, out_features=32, bias=True)
  (gat_layers): ModuleDict(
    (movieimage): ModuleList(
      (0-3): 4 x GATConv(
        (fc): Linear(in_features=32, out_features=128, bias=False)
        (feat_drop): Dropout(p=0.0, inplace=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (leaky_relu): LeakyReLU(negative_slope=0.2)
      )
    )
    (movietext): ModuleList(
      (0-3): 4 x GATConv(
        (fc): Linear(in_features=32, out_features=128, bias=False)
        (feat_drop): Dropout(

In [14]:
def test_model(model, test_loader, graph, features, k=5):
    device = next(model.parameters()).device  # 获取模型参数所在的设备
    model.eval()
    total_dcg = 0
    total_ndcg = 0
    total_recall = 0
    total_relevant_items = 0

    with torch.no_grad():
        for data in test_loader:
            user_ids, pos_item_ids, neg_item_ids = data
            user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)

            user_embeddings, pos_item_embeddings, _ = model(user_ids, pos_item_ids, graph, features)
            _, neg_item_embeddings, _ = model(user_ids, neg_item_ids, graph, features)

            pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
            neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)

            scores = torch.cat((pos_scores, neg_scores))
            labels = torch.cat((torch.ones_like(pos_scores), torch.zeros_like(neg_scores))).to(device)  # 真实标签

            # 计算分数并排序
            _, indices = torch.sort(scores, descending=True)
            sorted_labels = labels[indices]

            # 使用已定义的函数计算 DCG 和 NDCG
            dcg_value = dcg_at_k(sorted_labels, k)
            idcg_value = dcg_at_k(torch.ones(k).to(device), k)  # 理想情况下的排序
            ndcg_value = dcg_value / max(idcg_value, 1e-10)  # 避免除以零
            recall_value = sorted_labels[:k].sum() / labels.sum()

            total_dcg += dcg_value
            total_ndcg += ndcg_value
            total_recall += recall_value
            total_relevant_items += labels.sum()

    average_dcg = total_dcg / len(test_loader)
    average_ndcg = total_ndcg / len(test_loader)
    average_recall = total_recall / len(test_loader)
    print(f'Average DCG@{k}: {average_dcg:.4f}')
    print(f'Average NDCG@{k}: {average_ndcg:.4f}')
    print(f'Average Recall@{k}: {average_recall:.4f}')

In [15]:
# 加载模型
features = {
    'movieimage': torch.randn(5996, 2048),
    'movietext': torch.randn(5996, 384),
    'movieaudio': torch.randn(5996, 128)
}
model.load_state_dict(torch.load('D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth'))
model.eval()
test_model(model, valid_loader, graph, features)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

early stop

In [ ]:
def train_model(model, train_loader, lambda_reg, optimizer, device, validation_loader=None, patience=3):
    model.to(device)  # Ensure the model is on the correct device
    best_ndcg = 0.0
    epochs_no_improve = 0

    for epoch in range(100):  # Set a sufficiently large number for epochs
        model.train()
        total_loss = 0

        for user_ids, pos_item_ids, neg_item_ids in train_loader:
            user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)
            
            optimizer.zero_grad()
            user_embeddings, pos_item_embeddings = model(user_ids, pos_item_ids)
            _, neg_item_embeddings = model(user_ids, neg_item_ids)
            
            # Calculate BPR Loss
            pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
            neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
            loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-10))
            
            reg_loss = lambda_reg * (user_embeddings.norm(2).pow(2) +
                                     pos_item_embeddings.norm(2).pow(2) +
                                     neg_item_embeddings.norm(2).pow(2))
            total_loss_val = loss + reg_loss
            total_loss_val.backward()
            optimizer.step()
            total_loss += total_loss_val.item()

        print(f'Epoch {epoch+1}: Average Training Loss: {total_loss / len(train_loader)}')

        if validation_loader:
            _, average_ndcg, _ = evaluate_metrics(model, validation_loader)  # Assuming NDCG is the second returned value
            if average_ndcg > best_ndcg:
                best_ndcg = average_ndcg
                epochs_no_improve = 0
                # Optionally save the best model
                torch.save(model.state_dict(), 'model_best.pth')
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f'Early stopping triggered after {epoch+1} epochs.')
                    break


def evaluate_metrics(model, validation_loader, k=5):
    device = next(model.parameters()).device  # 获取模型参数所在的设备
    model.eval()
    total_dcg = 0
    total_ndcg = 0
    total_recall = 0
    total_relevant_items = 0

    with torch.no_grad():
        for data in validation_loader:
            user_ids, pos_item_ids, neg_item_ids = data
            user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)
            
            user_embeddings, pos_item_embeddings = model(user_ids, pos_item_ids)
            _, neg_item_embeddings = model(user_ids, neg_item_ids)
            
            pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
            neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
            
            scores = torch.cat((pos_scores, neg_scores))
            labels = torch.cat((torch.ones_like(pos_scores), torch.zeros_like(neg_scores)))  # Real labels
            
            # Calculate scores and sort them
            _, indices = torch.sort(scores, descending=True)
            sorted_labels = labels[indices]
            
            # Compute DCG and NDCG using predefined functions
            dcg_value = dcg_at_k(sorted_labels[:k], k)
            idcg_value = dcg_at_k(torch.ones(k).to(device), k)  # Ideal case sorting
            ndcg_value = dcg_value / max(idcg_value, 1e-10)  # Avoid division by zero
            recall_value = sorted_labels[:k].sum() / labels.sum()

            total_dcg += dcg_value
            total_ndcg += ndcg_value
            total_recall += recall_value
            total_relevant_items += labels.sum()

    average_dcg = total_dcg / len(validation_loader) if len(validation_loader) > 0 else 0
    average_ndcg = total_ndcg / len(validation_loader) if len(validation_loader) > 0 else 0
    average_recall = total_recall / total_relevant_items if total_relevant_items > 0 else 0

    print(f'Average DCG@{k}: {average_dcg:.4f}')
    print(f'Average NDCG@{k}: {average_ndcg:.4f}')
    print(f'Average Recall@{k}: {average_recall:.4f}')

    return average_dcg, average_ndcg, average_recall
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_sizes = {
    'movie': 2560,    # 如果电影节点存储所有多媒体特征的总和
    'actor': 50,      # 演员的嵌入维度
    'actress': 50,    # 女演员的嵌入维度
    'director': 50,   # 导演的嵌入维度
    'producer': 50,   # 制片的嵌入维度
    'movieimage': 2048,  # 图像特征维度
    'movietext': 384,    # 文字特征维度
    'movieaudio': 128,   # 音频特征维度
    'user': 100         # 假设用戶的嵌入维度，可能需要根据用户数量和系统复杂性进行调整
}
# 初始化和配置模型，加载数据，然后开始训练和评估
model = HeteroGraphConv(num_users, num_items, feature_sizes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, 0.01, optimizer, device, valid_loader, 7)
average_ndcg = evaluate_metrics(model, valid_loader)

# 保存模型
torch.save(model.state_dict(), 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth')

# 加载模型
model.load_state_dict(torch.load('D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth'))
model.eval()

In [ ]:
feature_sizes = {
    'movie': 2560,    # 如果电影节点存储所有多媒体特征的总和
    'actor': 50,      # 演员的嵌入维度
    'actress': 50,    # 女演员的嵌入维度
    'director': 50,   # 导演的嵌入维度
    'producer': 50,   # 制片的嵌入维度
    'movieimage': 2048,  # 图像特征维度
    'movietext': 384,    # 文字特征维度
    'movieaudio': 128,   # 音频特征维度
    'user': 100         # 假设用戶的嵌入维度，可能需要根据用户数量和系统复杂性进行调整
}
model = HeteroGraphConv(num_users, num_items, feature_sizes)
model.cuda()  # 将模型移至适当的设备（CPU或GPU）
optimizer = optim.Adam(model.parameters(), lr=0.001)
lambda_reg = 0.01  # 正则化权重，根据需要调整


# 保存模型
torch.save(model.state_dict(), 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth')

# 加载模型
model.load_state_dict(torch.load('D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth'))
model.eval()


In [ ]:
import torch.optim as optim


model = HeteroGraphConv(num_users, num_items, feature_sizes)
model.cuda()  # 将模型移至适当的设备（CPU或GPU）
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
print(df['userId'].max(), df['movieId'].max())  # 输出最大的 userId 和 movieId
user_item_pairs = list(zip(df['userId'].values, df['movieId'].values))
train_pairs, valid_pairs = train_test_split(user_item_pairs, test_size=0.2, random_state=42)
try:
    for i, (user_ids, pos_item_ids, neg_item_ids) in enumerate(train_loader):
        print(f"Batch {i}: {user_ids}, {pos_item_ids}, {neg_item_ids}")
except KeyError as e:
    print(f"KeyError with key: {e.args[0]}")
    # 这里可以添加更多的调试信息，如打印有问题的批次信息



In [ ]:
for user_ids, pos_item_ids, neg_item_ids in train_loader:
    user_ids, pos_item_ids, neg_item_ids = user_ids.cuda, pos_item_ids.cuda, neg_item_ids.cuda
    optimizer.zero_grad()
    # 接下来是模型的调用和损失计算


In [ ]:
for user_ids, pos_item_ids, neg_item_ids in train_loader:
    print(f"user_ids type: {user_ids.dtype}, shape: {user_ids.shape}")
    print(f"pos_item_ids type: {pos_item_ids.dtype}, shape: {pos_item_ids.shape}")
    print(f"neg_item_ids type: {neg_item_ids.dtype}, shape: {neg_item_ids.shape}")
    break  # 只查看第一批数据


In [ ]:
import torch
print(torch.__version__)


In [ ]:




# 保存模型
torch.save(model.state_dict(), 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model')



# Load the model (if needed)
model.load_state_dict(torch.load('D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model'))
model.eval()  # Set the model to evaluation mode




根據我們的程式

# 定义模型和优化器
model = HeteroGraphConv(feature_sizes)
optimizer = optim.Adam(model.parameters(), lr=0.001)
這樣寫是對的嗎?